# Last API with pylast

In [25]:
import pandas as pd 
import numpy as np 
import pylast
import os
import json

In [2]:
API_KEY = input()
API_SECRET = input()

network = pylast.LastFMNetwork(api_key=API_KEY, api_secret=API_SECRET)

### Getting users by network

In [3]:
def get_friends(user, limit = 50): 
    try: 
        friends = user.get_friends(limit = limit)
    except pylast.WSError: 
        return []
    name_friends = [link.name for link in friends]
    return name_friends 

def build_network(network, user, network_list = [], level = 5): 
    friends = get_friends(user)
    write_in_file(friends)
    network_list.extend(friends)
    if level > 0:
        for friend in friends: 
            build_network(network, network.get_user(friend), network_list, level - 1)
    else:
        return

def write_in_file(lst: list):
    with open('users.txt', 'a') as f: 
        for i in lst:
            f.write(i)
            f.write('\n')

#user = network.get_user(username = 'allanxguimaraes')
#write_in_file(['allanxguimaraes'])
#users = [] 
#build_network(network, user, users, level = 5)

## Get user information 

In [65]:
def get_track_id(track: str, artist: str):
    with open('id_tracks.json', 'r+') as f:
        data = json.load(f)
        try:
            track_id = data[track + ';;' + artist][0]
        except KeyError: 
            track_id = len(data.keys())
            data.update({track + ';;' + artist: track_id})
            print(data)
            f.seek(0)
            json.dump(data, f)
        return track_id

def get_track_by_id(track_id: int): 
    with open('id_tracks.json', 'r') as f:
        data = json.load(f)
    for t in data.items():
        if t[1] == track_id:
            return t[0].split(';;') 

In [66]:
def get_user_info(network, username: str) -> dict: 

    user = network.get_user(username = username)

    info = {}
    info['name'] = username
    info['country'] = user.get_country().name
    info['subscriber'] = int(user.is_subscriber())
    info['playcount'] = user.get_playcount()
    info['registered_since'] = user.get_registered()
    info['loved_tracks'] = []
    for loved in user.get_loved_tracks():
        track_id = get_track_id(loved.track.title, loved.track.artist.name)
        info['loved_tracks'].append(track_id)
    info['latest_tracks'] = []
    for latest in user.get_recent_tracks(limit = 50):
        track_id = get_track_id(latest.track.title, latest.track.artist.name)
        info['latest_tracks'].append(track_id)
    

In [38]:
user = network.get_user('allanxguimaraes')
print([i for i in user.__dir__() if i[0] != '_'])

['network', 'ws_prefix', 'name', 'get_name', 'get_artist_tracks', 'get_friends', 'get_loved_tracks', 'get_now_playing', 'get_recent_tracks', 'get_country', 'is_subscriber', 'get_playcount', 'get_registered', 'get_unixtime_registered', 'get_tagged_albums', 'get_tagged_artists', 'get_tagged_tracks', 'get_top_albums', 'get_top_artists', 'get_top_tags', 'get_top_tracks', 'get_track_scrobbles', 'get_image', 'get_url', 'get_library', 'get_wiki_published_date', 'get_wiki_summary', 'get_wiki_content', 'get_wiki', 'get_weekly_chart_dates', 'get_weekly_album_charts', 'get_weekly_artist_charts', 'get_weekly_track_charts', 'get_weekly_charts']


In [30]:
with open('users.txt', 'r') as f:
    text = f.read()
    users = text.split(sep = '\n')

users = np.unique(users)
chosen_users = np.random.choice(users, size = 100)

users_info = []
for username in chosen_users:
    user = network.get_user(username = username)